In [0]:
%sql

USE Logistics_db;

-- Drop and recreate Dim_S
CREATE OR REPLACE TABLE Dim_Ship USING DELTA AS 
SELECT `Year` AS ShipmentsYear, Mode, Units, Value, Statistic AS StatisticType 
FROM Shipments;

-- Drop and recreate Dim_B
CREATE OR REPLACE TABLE Dim_Bord USING DELTA AS 
SELECT PortName, State, Border, Measure, PortCode, `Date` AS BorderYear 
FROM borders;

-- Drop and recreate Dim_P
CREATE OR REPLACE TABLE Dim_Potr USING DELTA AS 
SELECT DISTINCT PortID, PortName, CargoType, Region, TradeType, PortRanking, PercentChange 
FROM ports;

CREATE OR REPLACE TABLE Facts_Trade AS 
SELECT 
    p.PortID, 
    s.ShipmentsYear,  
    p.TradeType, 
    p.PortRanking, 
    p.PercentChange, 
    s.Units, 
    b.Border
FROM Dim_Bord b
JOIN Dim_Ship s ON RIGHT(b.BorderYear, 4) = s.ShipmentsYear
JOIN Dim_Potr p ON p.PortID = b.PortCode;

CREATE OR REPLACE TABLE Facts_Port_Borders USING DELTA AS 
SELECT 
    b.PortName,
    b.State,
    b.Border,
    p.Region,
    p.PortRanking
FROM Dim_Bord b JOIN  Dim_Potr p on b.PortCode = p.PortID;



num_affected_rows,num_inserted_rows
